In [3]:
import nltk

# aulas anteriores

Vamos começar com um texto simples

In [4]:
# passo 1: carregando o corpus

texto = """No meio do caminho tinha uma pedra
tinha uma pedra no meio do caminho
tinha uma pedra
no meio do caminho tinha uma pedra."""
texto = texto.lower().split('\n')

In [5]:
# passo 2: tokenizando as sentenças

texto_tok = [nltk.tokenize.word_tokenize(verso, language='portuguese') for verso in texto]
texto_tok

[['no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra'],
 ['tinha', 'uma', 'pedra', 'no', 'meio', 'do', 'caminho'],
 ['tinha', 'uma', 'pedra'],
 ['no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', '.']]

In [6]:
from nltk.tokenize import wordpunct_tokenize
tmp = [nltk.tokenize.wordpunct_tokenize(verso) for verso in texto]
tmp

[['no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra'],
 ['tinha', 'uma', 'pedra', 'no', 'meio', 'do', 'caminho'],
 ['tinha', 'uma', 'pedra'],
 ['no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', '.']]

# modelos de linguagem

Começamos aqui...

1. Veja slides
2. https://web.stanford.edu/~jurafsky/slp3/3.pdf

In [7]:
# passo 3: inserindo marcadores de início e fim

from nltk.lm.preprocessing import pad_both_ends
texto_tok_pad = [list(pad_both_ends(v,2)) for v in texto_tok]
texto_tok_pad

[['<s>', 'no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', '</s>'],
 ['<s>', 'tinha', 'uma', 'pedra', 'no', 'meio', 'do', 'caminho', '</s>'],
 ['<s>', 'tinha', 'uma', 'pedra', '</s>'],
 ['<s>', 'no', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', '.', '</s>']]

In [8]:
# passo 4: calculando n-grams

from nltk.util import everygrams
ngrams_pad = [list(everygrams(v,max_len=2)) for v in texto_tok_pad]
ngrams_pad

[[('<s>',),
  ('<s>', 'no'),
  ('no',),
  ('no', 'meio'),
  ('meio',),
  ('meio', 'do'),
  ('do',),
  ('do', 'caminho'),
  ('caminho',),
  ('caminho', 'tinha'),
  ('tinha',),
  ('tinha', 'uma'),
  ('uma',),
  ('uma', 'pedra'),
  ('pedra',),
  ('pedra', '</s>'),
  ('</s>',)],
 [('<s>',),
  ('<s>', 'tinha'),
  ('tinha',),
  ('tinha', 'uma'),
  ('uma',),
  ('uma', 'pedra'),
  ('pedra',),
  ('pedra', 'no'),
  ('no',),
  ('no', 'meio'),
  ('meio',),
  ('meio', 'do'),
  ('do',),
  ('do', 'caminho'),
  ('caminho',),
  ('caminho', '</s>'),
  ('</s>',)],
 [('<s>',),
  ('<s>', 'tinha'),
  ('tinha',),
  ('tinha', 'uma'),
  ('uma',),
  ('uma', 'pedra'),
  ('pedra',),
  ('pedra', '</s>'),
  ('</s>',)],
 [('<s>',),
  ('<s>', 'no'),
  ('no',),
  ('no', 'meio'),
  ('meio',),
  ('meio', 'do'),
  ('do',),
  ('do', 'caminho'),
  ('caminho',),
  ('caminho', 'tinha'),
  ('tinha',),
  ('tinha', 'uma'),
  ('uma',),
  ('uma', 'pedra'),
  ('pedra',),
  ('pedra', '.'),
  ('.',),
  ('.', '</s>'),
  ('</s>',)]]

In [9]:
# passo 5: colocando tokens em lista única

from nltk.lm.preprocessing import flatten
tokens = list(flatten(texto_tok_pad))
tokens

['<s>',
 'no',
 'meio',
 'do',
 'caminho',
 'tinha',
 'uma',
 'pedra',
 '</s>',
 '<s>',
 'tinha',
 'uma',
 'pedra',
 'no',
 'meio',
 'do',
 'caminho',
 '</s>',
 '<s>',
 'tinha',
 'uma',
 'pedra',
 '</s>',
 '<s>',
 'no',
 'meio',
 'do',
 'caminho',
 'tinha',
 'uma',
 'pedra',
 '.',
 '</s>']

In [10]:
# passo 6: definindo vocabulário

from nltk.lm import Vocabulary
vocab = Vocabulary(tokens, unk_cutoff=1)
list(vocab)

['<s>',
 'no',
 'meio',
 'do',
 'caminho',
 'tinha',
 'uma',
 'pedra',
 '</s>',
 '.',
 '<UNK>']

In [11]:
# passo 7: treinando o modelo de linguagem (3,4,5,6)

from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

ngrams_pad, vocab = padded_everygram_pipeline(2, texto_tok)
lm2 = MLE(2)
lm2.fit(ngrams_pad, vocab)

ngrams_pad, vocab = padded_everygram_pipeline(1, texto_tok)
lm1 = MLE(1)
lm1.fit(ngrams_pad, vocab)

In [16]:
lm2.generate(5, text_seed=["<s>","no"]) # comparar com lm1

['meio', 'do', 'caminho', 'tinha', 'uma']

In [17]:
lm2.score('no'), lm2.logscore('no')

(0.09090909090909091, -3.4594316186372978)

In [22]:
lm2.score('tinha', context = ['caminho']), lm2.logscore('tinha',context = ['caminho'])

(0.6666666666666666, -0.5849625007211563)

# avaliando o modelo

1. extrinsic
2. intrinsic (perplexity)

Para a avaliação intrínseca do ML, precisamos de um conjunto de teste. As probabilidades do ML de n-gram vieram do corpus em que é treinado, o conjunto de treinamento ou corpus de treinamento. Medimos a qualidade do modelo pelo seu desempenho em alguns dados não vistos chamados de conjunto de teste ou corpus de teste.

a melhoria de um modelo em perplexidade deve sempre ser confirmada por uma avaliação de ponta a ponta, em uma tarefa real, antes de concluir a avaliação do modelo.

In [70]:
teste = """Tinha uma pedra
No meio do caminho
Tinha uma pedra."""

teste = teste.lower().split('\n')
teste_tok = [ nltk.word_tokenize(v, language='portuguese') for v in teste]

teste_ng, _ = padded_everygram_pipeline(1, teste_tok)
teste_ng = flatten(list(w) for w in teste_ng)

# lm to texto
p = lm1.perplexity(teste_ng)  
print(f"perplexidade do unigrama: {p}")

teste_ng, _ = padded_everygram_pipeline(2, teste_tok)
teste_ng = flatten(list(w) for w in teste_ng)

# lm to texto
p = lm2.perplexity(teste_ng)
print(f"perplexidade do bigrama: {p}")

perplexidade do unigrama: 7.8712736589589865
perplexidade do bigrama: 4.090428596027453


# generalização

1. Laplace add-1 smoothing
2. Lidstone

In [78]:
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import Laplace

ngrams_pad, vocab = padded_everygram_pipeline(2, texto_tok)
lm = Laplace(2)
lm.fit(ngrams_pad, vocab)
lm.score("tinha", context=["caminho"]) # compare com valor anterior

0.21428571428571427

In [79]:
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import Lidstone

ngrams_pad, vocab = padded_everygram_pipeline(2, texto_tok)
lm = Lidstone(order=2, gamma = 0.1)
lm.fit(ngrams_pad, vocab)
lm.score("tinha", context=["caminho"]) # compare com valor anterior

0.5121951219512195